In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def generate_dataset(X,Y,
        x0,y0,R,
        error=0.3,drop_rate=0.1
        ):
    df=pd.DataFrame({'x':X,'y':Y})
    df_in_circle=df[(df['x']-x0)**2+(df['y']-y0)**2<R**2]
    df_in_circle.loc[:, 'x'] += np.random.uniform(-error, error, len(df_in_circle))
    df_in_circle.loc[:, 'y'] += np.random.uniform(-error, error, len(df_in_circle))
    df_in_circle=df_in_circle.sample(frac=1-drop_rate)
    return df_in_circle

def transform_dataset(x0,y0,theta,df_in_circle):
    # 平移df_in_circle到原点
    df_in_circle.loc[:, 'x'] -= x0
    df_in_circle.loc[:, 'y'] -= y0
    # 绕圆心旋转df_in_circle中的点theta角
    x_new=df_in_circle['x']*np.cos(theta)-df_in_circle['y']*np.sin(theta)
    y_new=df_in_circle['x']*np.sin(theta)+df_in_circle['y']*np.cos(theta)
    df_in_circle['x']=x_new
    df_in_circle['y']=y_new
    return df_in_circle


In [7]:
filename='axicon_pattern_crossflower.csv'
df = pd.read_csv(filename)
X=df['x']
Y=df['y']

x0,y0=np.random.uniform(-20,20,2)
measure_diameter=17
theta=np.random.uniform(0,2*np.pi)
theta=0

df_in_circle=generate_dataset(X,Y,x0,y0,measure_diameter/2,error=0.1)
df_in_circle=transform_dataset(x0,y0,theta,df_in_circle)
df_in_circle.to_csv('axicon_pattern_crossflower_measure.csv',index=False)

data_measured=df_in_circle.values
data_original=df.values

In [8]:
import numpy as np
from scipy.spatial.distance import cdist

def estimate_affine_matrix(source, target):
    """使用最小二乘法估计仿射变换矩阵"""
    A = np.zeros((2 * len(source), 6))
    b = np.zeros((2 * len(source)))
    for i in range(len(source)):
        A[2 * i] = [source[i][0], source[i][1], 1, 0, 0, 0]
        A[2 * i + 1] = [0, 0, 0, source[i][0], source[i][1], 1]
        b[2 * i] = target[i][0]
        b[2 * i + 1] = target[i][1]
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    return x.reshape(2, 3)

def apply_transform(points, transform):
    """应用仿射变换到点集"""
    return np.dot(np.hstack((points, np.ones((points.shape[0], 1)))), transform.T)[:,:2]

def random_partition(n, n_data):
    """随机选择 n 个数据点用于模型估计"""
    all_idxs = np.arange(n_data)
    np.random.shuffle(all_idxs)
    idxs1 = all_idxs[:n]
    idxs2 = all_idxs[n:]
    return idxs1, idxs2

def ransac(data_original, data_measured, model, n, k, t, d):
    best_fit = None
    best_err = np.inf
    best_inlier_idxs = None
    
    for _ in range(k):
        # 随机选择 n 个点
        maybe_idxs, _ = random_partition(n, len(data_measured))
        maybe_inliers = data_measured[maybe_idxs]
        test_idxs = [i for i in range(len(data_measured)) if i not in maybe_idxs]
        test_points = data_measured[test_idxs]
        
        # 使用模型估计变换
        maybe_model = model(maybe_inliers, data_original)
        
        # 应用变换到所有测量点
        test_transformed = apply_transform(test_points, maybe_model)
        
        # 计算内点数
        distances = cdist(test_transformed, data_original)
        inliers = distances.min(axis=1) < t
        
        inlier_count = np.sum(inliers)
        if inlier_count > d and inlier_count > best_inlier_idxs:
            best_inlier_idxs = inlier_count
            best_fit = maybe_model
            best_err = distances.min(axis=1).mean()
    
    return best_fit, best_inlier_idxs

# 使用示例
ransac_model = lambda a, b: estimate_affine_matrix(a, b)
# 确保 data_measured 和 data_original 是已经加载的 NumPy 数组
init_transform, inliers = ransac(data_original, data_measured, ransac_model, 10, 1000, 1.0, 100)

print("初始变换矩阵:", init_transform)
print("内点数量:", inliers)


初始变换矩阵: None
内点数量: None
